In [1]:
import argparse
import os
import sys
import time
import math
import numpy as np
import torch
import torch.nn as nn
#from torch.utils.data import Dataset, DataLoader
#import torch.utils.data
import torch.nn.functional as F
import torch.cuda as cutorch


import gc

import random
import model as model_code
import nsd_loss

from utils import seed_all_randomness, create_exp_dir, save_checkpoint, load_idx2word_freq, load_emb_file_to_tensor, load_corpus, output_parallel_models, str2bool

In [2]:
class arguments():
    def __init__(self):
        # path
        self.data = '/iesl/canvas/hanqingli/NSD_for_sentence_embedding/data/wiki2016_min100/5_5/'
        self.tensor_folder='corpus_00'
        self.training_file='train.pt'
        self.save='/iesl/canvas/hanqingli/NSD_for_sentence_embedding/saves/test1'
        self.emb_file='/iesl/canvas/hschang/language_modeling/NSD_for_sentence_embedding/resources/glove.840B.300d_filtered_wiki2016.txt'
        # parser.add_argument('--stop_word_file', type=str, default='./resources/stop_word_list',
        #                    help='path to the file of a stop word list')

        # encoder
        # both
#         parser.add_argument('--en_model', type=str, default='LSTM',
#                             help='type of encoder model (LSTM, LSTM+TRANS, TRANS+LSTM, TRANS)')
        self.en_model='TRANS'
#         parser.add_argument('--emsize', type=int, default=0,
#                             help='size of word embeddings')
        self.emsize=0
#         parser.add_argument('--dropouti', type=float, default=0.4,
#                             help='dropout for input embedding layers (0 = no dropout)')
        self.dropouti=0.1
#         parser.add_argument('--dropoute', type=float, default=0.1,
#                             help='dropout to remove words from embedding layer (0 = no dropout)')
        self.dropoute=0
        # LSTM only
#         parser.add_argument('--nhid', type=int, default=600,
#                             help='number of hidden units per layer in LSTM')
        self.nhid=600
#         parser.add_argument('--nlayers', type=int, default=1,
#                             help='number of layers')
        self.nlayers=1
#         parser.add_argument('--dropout', type=float, default=0.4,
#                             help='dropout applied to the output layer (0 = no dropout)')
        self.dropout=0
        # TRANS only
#         parser.add_argument('--encode_trans_layers', type=int, default=5,
#                             help='How many layers we have in transformer. Do not have effect if de_model is LSTM')
        self.encode_trans_layers=5
#         parser.add_argument('--trans_nhid', type=int, default=-1,
#                             help='number of hidden units per layer in transformer')
        self.trans_nhid=-1
#         parser.add_argument('--nhidlast', type=int, default=-1,
#                             help='number of hidden units for the last rnn layer')
        self.nhidlast=-1
#         parser.add_argument('--dropouth', type=float, default=0.3,
#                             help='dropout for rnn layers (0 = no dropout)')
        self.dropouth=0.3
#         parser.add_argument('--dropoutl', type=float, default=-0.2,
#                             help='dropout applied to layers (0 = no dropout)')
        self.dropoutl=-0.25
#         parser.add_argument('--wdrop', type=float, default=0.5,
#                             help='amount of weight dropout to apply to the RNN hidden to hidden matrix')
        self.wdrop=0.5

        # decoder
        # both
#         parser.add_argument('--de_model', type=str, default='LSTM',
#                             help='type of decoder model (LSTM, LSTM+TRANS, TRANS+LSTM, TRANS)')
#         parser.add_argument('--de_coeff_model', type=str, default='LSTM',
#                             help='type of decoder model to predict coefficients (LSTM, TRANS)')
#         parser.add_argument('--n_basis', type=int, default=10,
#                             help='number of basis we want to predict')
#         # parser.add_argument('--linear_mapping_dim', type=int, default=0,
#         #                    help='map the input embedding by linear transformation')
#         parser.add_argument('--positional_option', type=str, default='linear',
#                             help='options of encode positional embedding into models (linear, cat, add)')
#         parser.add_argument('--dropoutp', type=float, default=0.5,
#                             help='dropout of positional embedding or input embedding after linear transformation (when linear_mapping_dim != 0)')
#         # LSTM only
#         parser.add_argument('--nhidlast2', type=int, default=-1,
#                             help='hidden embedding size of the second LSTM')
#         # TRANS only
#         parser.add_argument('--trans_layers', type=int, default=5,
#                             help='How many layers we have in transformer. Do not have effect if de_model is LSTM')
#         parser.add_argument('--de_en_connection', type=str2bool, nargs='?', default=True,
#                             help='If True, using Transformer decoder in our decoder. Otherwise, using Transformer encoder')
#         parser.add_argument('--dropout_prob_trans', type=float, default=0.1,
#                             help='hidden_dropout_prob and attention_probs_dropout_prob in Transformer')
        # coeff
#         parser.add_argument('--w_loss_coeff', type=float, default=0.1,
#                             help='weights for coefficient prediction loss')
        self.w_loss_coeff=0.1
#         parser.add_argument('--L1_losss_B', type=float, default=0.2,
#                             help='L1 loss for the coefficient matrix')
        self.L1_losss_B=0.25
        # parser.add_argument('--coeff_opt', type=str, default='max',
#         parser.add_argument('--coeff_opt', type=str, default='lc',
#                             help='Could be max, lc, maxlc')
        self.coeff_opt='lc'
#         parser.add_argument('--coeff_opt_algo', type=str, default='rmsprop',
#                             # parser.add_argument('--coeff_opt_algo', type=str, default='sgd_bmm',
#                             help='Could be sgd_bmm, sgd, asgd, adagrad, rmsprop, and adam')
        self.coeff_opt_algo='rmsprop'
        # target emb
#         parser.add_argument('--update_target_emb', default=False, action='store_true',
#                             help='Whether to update target embedding')
        self.update_target_emb=False
#         parser.add_argument('--target_emb_source', type=str, default='ext',
#                             help='Could be ext (external), rand or ewe (encode word embedding)')
        self.target_emb_source='ext'

        # training
#         parser.add_argument('--optimizer', type=str, default="SGD",
#                             help='optimization algorithm. Could be SGD or Adam')
        self.optimizer='SGD'
#         parser.add_argument('--lr', type=float, default=1,
#                             help='initial learning rate')
        self.lr=1.0
#         parser.add_argument('--lr2_divide', type=float, default=1.0,
#                             help='drop this ratio for the learning rate of the second LSTM')
        self.lr2_divide=1.0
#         parser.add_argument('--clip', type=float, default=0.25,
#                             help='gradient clipping')
        self.clip=0.25
#         parser.add_argument('--epochs', type=int, default=100,
#                             help='upper epoch limit')
        self.epochs=100
#         parser.add_argument('--batch_size', type=int, default=200, metavar='N',
#                             help='batch size')
        self.batch_size=200
#         parser.add_argument('--small_batch_size', type=int, default=-1,
#                             help='the batch size for computation. batch_size should be divisible by small_batch_size.\
#                              In our implementation, we compute gradients with small_batch_size multiple times, and accumulate the gradients\
#                              until batch_size is reached. An update step is then performed.')
        self.small_batch_size=-1
#         parser.add_argument('--wdecay', type=float, default=1e-6,
#                             help='weight decay applied to all weights')
        self.wdecay=1e-6
#         parser.add_argument('--nonmono', type=int, default=1,
#                             help='decay learning rate after seeing how many validation performance drop')
        self.nonmono=1
#         parser.add_argument('--training_split_num', type=int, default=2,
#                             help='We want to split training corpus into how many subsets. Splitting training dataset seems to make pytorch run much faster and we can store and eval the model more frequently')
        self.training_split_num=200
#         parser.add_argument('--valid_per_epoch', type=int, default=2,
#                             help='Number of times we want to run through validation data and save model within an epoch')
        self.valid_per_epoch=200
#         parser.add_argument('--copy_training', type=str2bool, nargs='?', default=True,
#                             help='turn off this option to save some cpu memory when loading training data')
        self.copy_training=True

        # system
#         parser.add_argument('--seed', type=int, default=1111,
#                             help='random seed')
        self.seed=1111
#         parser.add_argument('--cuda', type=str2bool, nargs='?', default=True,
#                             help='use CUDA')
        self.cuda=True
#         parser.add_argument('--single_gpu', default=False, action='store_true',
#                             help='use single GPU')
        self.single_gpu=False
#         parser.add_argument('--log-interval', type=int, default=200, metavar='N',
#                             help='report interval')
#         self.log-interval=200
#         parser.add_argument('--continue_train', action='store_true',
#                             help='continue train from a checkpoint')
        self.continue_train=False
#         parser.add_argument('--start_training_split', type=int, default=0,
#                             help='We want to split training corpus into how many subsets. Splitting training dataset seems to make pytorch run much faster and we can store and eval the model more frequently')
        self.start_training_split=0

args = arguments()


In [3]:
########################
print("Set up environment")
########################
assert args.training_split_num >= args.valid_per_epoch

if not args.continue_train:
    args.save = '{}-{}'.format(args.save, time.strftime("%Y%m%d-%H%M%S"))
create_exp_dir(args.save, scripts_to_save=['main.ipynb', 'model.py', 'nsd_loss.py'])


Set up environment
Experiment dir : /iesl/canvas/hanqingli/NSD_for_sentence_embedding/saves/test1-20200310-171353


In [4]:
if args.coeff_opt == 'maxlc':
    current_coeff_opt = 'max'
else:
    current_coeff_opt = args.coeff_opt

#if args.dropoutl < 0:
#    args.dropoutl = args.dropouth
if args.small_batch_size < 0:
    args.small_batch_size = args.batch_size


assert args.batch_size % args.small_batch_size == 0, 'batch_size must be divisible by small_batch_size'


In [5]:
def logging(s, print_=True, log_=True):
    if print_:
        print(s)
        sys.stdout.flush()
    if log_:
        with open(os.path.join(args.save, 'log.txt'), 'a+') as f_log:
            f_log.write(s + '\n')
            
# Set the random seed manually for reproducibility.
seed_all_randomness(args.seed,args.cuda)

logging('Args: {}'.format(args))

Args: <__main__.arguments object at 0x7f2fdfbbdb50>


In [6]:
print("Loading data")

device = torch.device("cuda" if args.cuda else "cpu")

idx2word_freq, dataloader_train_arr, dataloader_val, dataloader_val_shuffled, max_sent_len = load_corpus(args.data, args.batch_size, args.batch_size, device, args.tensor_folder, args.training_file, args.training_split_num, args.copy_training)


def counter_to_tensor(idx2word_freq,device):
    total = len(idx2word_freq)
    w_freq = torch.zeros(total, dtype=torch.float, device = device, requires_grad = False)
    for i in range(total):
        w_freq[i] = 1
        #w_freq[i] = math.sqrt(idx2word_freq[x][1])
        #w_freq[i] = idx2word_freq[x][1]
    w_freq[0] = -1
    return w_freq

external_emb = torch.tensor([0.])
extra_init_idx = []
if len(args.emb_file) > 0:
    #with torch.no_grad():
    external_emb, output_emb_size, extra_init_idx = load_emb_file_to_tensor(args.emb_file, device, idx2word_freq)
    external_emb = external_emb / (0.000000000001 + external_emb.norm(dim = 1, keepdim=True))
    external_emb.requires_grad = args.update_target_emb
    print("loading ", args.emb_file)
else:
    if args.target_emb_source == 'ewe':
        output_emb_size = args.emsize
        print("Using word embedding from encoder")
    elif args.target_emb_source == 'rand' and args.update_target_emb == True:
        output_emb_size = args.emsize
        external_emb = torch.randn(len(idx2word_freq), output_emb_size, device = device, requires_grad = False)
        external_emb = external_emb / (0.000000000001 + external_emb.norm(dim = 1, keepdim=True))
        external_emb.requires_grad = True
        print("Initialize target embedding randomly")
    else:
        print("we don't support such target_emb_source " + args.target_emb_source + ", update_target_emb ", args.update_target_emb, ", and emb_file "+ args.emb_file)
        sys.exit(1)

if args.trans_nhid < 0:
    if args.emsize > 0:
        args.trans_nhid = args.emsize
    else:
        args.trans_nhid = output_emb_size


w_freq = counter_to_tensor(idx2word_freq,device)

Loading data
OOV word type percentage: 10.343283933746436%
OOV token percentage: 0.748010626181413%
loading  /iesl/canvas/hschang/language_modeling/NSD_for_sentence_embedding/resources/glove.840B.300d_filtered_wiki2016.txt


In [7]:
########################
print("Building models")
########################

ntokens = len(idx2word_freq)

encoder = model_code.SEQ2EMB(args.en_model.split('+'), ntokens, args.emsize, args.nhid, args.nlayers,
               args.dropout, args.dropouti, args.dropoute, max_sent_len,  external_emb, extra_init_idx, args.encode_trans_layers, args.trans_nhid)
criterion = nn.MSELoss().cuda()
# if args.nhidlast2 < 0:
#     #args.nhidlast2 = output_emb_size
#     args.nhidlast2 = encoder.output_dim

Building models
Randomly initializes embedding for  30730  words


In [8]:
import torch.nn.init as weight_init
def initialize_weights(net, normal_std):
    for name, param in net.named_parameters(): 
        if 'bias' in name or 'rnn' not in name:
            #print("skip "+name)
            continue
        print("normal init "+name+" with std"+str(normal_std) )
        weight_init.normal_(param, std = normal_std)

In [9]:
if args.continue_train:
    encoder.load_state_dict(torch.load(os.path.join(args.save, 'encoder.pt')))
    # decoder.load_state_dict(torch.load(os.path.join(args.save, 'decoder.pt')))

In [10]:
parallel_encoder = output_parallel_models(args.cuda, args.single_gpu, encoder)

total_params = sum(x.data.nelement() for x in encoder.parameters())
logging('Encoder total parameters: {}'.format(total_params))

Encoder total parameters: 94580450


In [34]:
########################
print("Training")
########################

def evaluate(dataloader, external_emb, current_coeff_opt):
    # Turn on evaluation mode which disables dropout.
    encoder.eval()
    # decoder.eval()
    total_loss = 0
    total_loss_set = 0
    with torch.no_grad():
        for i_batch, sample_batched in enumerate(dataloader):
            feature, target = sample_batched
            output_emb_last, _ = parallel_encoder(feature)

            compute_target_grad = False
            loss = criterion(output_emb_last, target)
            total_loss += loss * args.batch_size
            # total_loss_set += loss_set * batch_size

    return total_loss / len(dataloader.dataset)


Training


In [35]:
def train_one_epoch(dataloader_train, external_emb, lr, current_coeff_opt, split_i):
    start_time = time.time()
    total_loss = 0.
    total_loss_set = 0.
    total_loss_set_reg = 0.
    total_loss_set_div = 0.
    total_loss_set_neg = 0.
    total_loss_coeff_pred = 0.

    encoder.train()
    # decoder.train()
    for i_batch, sample_batched in enumerate(dataloader_train):
        feature, target = sample_batched
        optimizer_e.zero_grad()
        output_emb_last, _ = parallel_encoder(feature)
        target = target.type(torch.cuda.FloatTensor)
        loss = criterion(output_emb_last, target)

        loss *= float(args.small_batch_size / args.batch_size)
        total_loss += loss.item()

        loss.backward()

        gc.collect()

        torch.nn.utils.clip_grad_norm_(encoder.parameters(), args.clip)
        # torch.nn.utils.clip_grad_norm_(decoder.parameters(), args.clip)
        optimizer_e.step()
        if len(args.emb_file) == 0 and args.target_emb_source == 'ewe':
            encoder.encoder.weight.data[0, :] = 0

        # optimizer_d.step()

        if args.update_target_emb:
            #print(external_emb.requires_grad)
            #print(external_emb.grad)
            if args.optimizer == 'SGD':
                external_emb.data -= lr/args.lr2_divide/10.0 * external_emb.grad.data
            else:
                #external_emb.data -= 0.1/args.lr2_divide/10.0 * external_emb.grad.data
                external_emb.data -= 10/args.lr2_divide/10.0 * external_emb.grad.data
            if args.target_emb_source != 'ewe':
                external_emb.data[0,:] = 0
            external_emb.grad.data.zero_()
            #with torch.no_grad():
            external_emb.data = external_emb.data / (0.000000000001 + external_emb.data.norm(dim = 1, keepdim=True))

        if i_batch % 200 == 0 and i_batch > 0:
            cur_loss = total_loss / 200
            elapsed = time.time() - start_time
            logging('| e {:3d} {:3d} | {:5d}/{:5d} b | lr {:02.2f} | ms/batch {:5.2f} | '
                    'l {:5.2f}'.format(
                epoch, split_i, i_batch, len(dataloader_train.dataset) // args.batch_size, optimizer_e.param_groups[0]['lr'],
                elapsed * 1000 / 200, cur_loss))
            #if args.coeff_opt == 'maxlc' and current_coeff_opt == 'max' and cur_loss_set + cur_loss_set_neg < -0.02:
            if args.coeff_opt == 'maxlc' and current_coeff_opt == 'max':
                current_coeff_opt = 'lc'
                print("switch to lc")
            total_loss = 0.
            start_time = time.time()
            break
    return current_coeff_opt

In [36]:
if args.optimizer == 'SGD':
    optimizer_e = torch.optim.SGD(encoder.parameters(), lr=args.lr, weight_decay=args.wdecay)
    # optimizer_d = torch.optim.SGD(decoder.parameters(), lr=args.lr/args.lr2_divide, weight_decay=args.wdecay)
else:
    optimizer_e = torch.optim.Adam(encoder.parameters(), lr=args.lr, weight_decay=args.wdecay)
    # optimizer_d = torch.optim.Adam(decoder.parameters(), lr=args.lr/args.lr2_divide, weight_decay=args.wdecay)


In [37]:
if args.continue_train:
    optimizer_e_state_dict = torch.load(os.path.join(args.save, 'optimizer_e.pt'), map_location=device)
    optimizer_e.load_state_dict(optimizer_e_state_dict)
    # optimizer_d_state_dict = torch.load(os.path.join(args.save, 'optimizer_d.pt'), map_location=device)
    # optimizer_d.load_state_dict(optimizer_d_state_dict)


In [39]:
lr = args.lr
best_val_loss = None
nonmono_count = 0
saving_freq = int(math.floor(args.training_split_num / args.valid_per_epoch))

for epoch in range(1, args.epochs+1):
    epoch_start_time = time.time()
    for i in range(len(dataloader_train_arr)):
        if epoch == 1 and i < args.start_training_split:
            print("Skipping epoch "+str(epoch) + ' split '+str(i) )
            continue
        current_coeff_opt = train_one_epoch(dataloader_train_arr[i], external_emb, lr, current_coeff_opt, i)
        
        if i != args.training_split_num - 1 and (i + 1) % saving_freq != 0:
            continue

        val_loss_all = evaluate(dataloader_val, external_emb, current_coeff_opt)
        print(val_loss_all)
        logging('-' * 89)
        logging('| end of epoch {:3d} split {:3d} | time: {:5.2f}s | lr {:5.2f} | valid loss {:5.2f}'
                .format(epoch, i, (time.time() - epoch_start_time), lr,
                                           val_loss_all))
        
        val_loss_important = val_loss_all
        
        logging('-' * 89)
        #dataloader_val_shuffled?
        val_loss_all = evaluate(dataloader_val_shuffled, external_emb, current_coeff_opt)
        logging('-' * 89)
        logging('| Shuffled | time: {:5.2f}s | lr {:5.2f} | valid loss {:5.2f}'
                .format((time.time() - epoch_start_time), lr,
                                           val_loss_all))
        logging('-' * 89)
        
        if not best_val_loss or val_loss_important < best_val_loss:
            # save_checkpoint(encoder, decoder, optimizer_e, optimizer_d, external_emb, args.save)
            save_checkpoint(encoder, optimizer_e, external_emb, args.save)
            best_val_loss = val_loss_important
            logging('Models Saved')
        else:
            nonmono_count += 1
        
        if nonmono_count >= args.nonmono:
            # Anneal the learning rate if no improvement has been seen in the validation dataset.
            nonmono_count = 0
            lr /= 4.0
            for param_group in optimizer_e.param_groups:
                param_group['lr'] = lr
            # for param_group in optimizer_d.param_groups:
            #     param_group['lr'] = lr/args.lr2_divide

/home/hanqingli/workspace/NSD_for_sentence_embedding/src/utils.py:131: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  feature = torch.tensor(self.feature[idx, :], dtype = torch.long, device = self.output_device)
/home/hanqingli/workspace/NSD_for_sentence_embedding/src/utils.py:135: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(self.target[idx, :], dtype = torch.long, device = self.output_device)


| e   1   0 |   200/  217 b | lr 1.00 | ms/batch 372.88 | l  0.44


NameError: name 'val_loss_alll' is not defined

In [ ]:
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            if 'cuda' in str(obj.device):
                print(obj.device)
                del obj
                torch.cuda.empty_cache()
    except:
        pass